In [ ]:
%pip install folium
# restart kernel after installation

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import duckdb
conn = duckdb.connect()

In [ ]:
conn.execute('''
  CREATE TABLE flights
    as
    SELECT
    *
  FROM read_parquet('./datasets/flights/year=2015/month=??/flights.parquet')
''')

In [ ]:
conn.execute('''
  CREATE TABLE airports
    as
    SELECT
      *
    FROM read_csv_auto('./datasets/flights/airports.csv')
''')

In [ ]:
conn.execute('''
  CREATE TABLE airlines
    as
    SELECT
    *
    FROM read_csv('./datasets/flights/airlines.csv')
''')

In [ ]:
display(conn.execute('SHOW TABLES').df())

,name
0,airlines
1,airports
2,flights


In [ ]:
import folium
mymap = folium.Map(location = [47.116386, -101.299591],
width = 950,
height = 550,
zoom_start = 3,
tiles = 'openstreetmap')
mymap       
# mymap.save('mymap.html')

# To view the output in your browser from a Github Codespace, run this in a terminal:
# $BROWSER mymap.html

In [ ]:
import folium

mymap = folium.Map(location = [47.116386, -101.299591],
                   width = 950,
                   height = 550,
                   zoom_start = 3,
                   tiles = 'openstreetmap')

folium.TileLayer('cartodbpositron',
                 attr = 'cartodbpositron',
                 show = False).add_to(mymap)

folium.TileLayer('cartodbdark_matter',
                 attr = 'cartodbdark_matter',
                 show = False).add_to(mymap)

folium.LayerControl().add_to(mymap)
mymap
# mymap.save('mymap.html')

In [ ]:
df = conn.execute('''
  SELECT
    latitude as lat,
    longitude as lng,
    airport as airport
  FROM airports
  WHERE
    (lat is not null) or
    (lng is not null)
''').df()

In [ ]:
import math

for lat, lng, airport in zip(df['lat'], df['lng'], df['airport']):
    airport = folium.CircleMarker(
        location = [lat, lng], # location of the marker
        radius = 4,            # size of the marker
        color = 'red',         # color of the marker
        fill = True,           # fill the marker with color
        fill_color = 'yellow', # fill the marker with yellow color
        fill_opacity = 0.5,    # make the marker translucent
        popup = airport)       # name of the airport
    
    # add the circle marker to the map
    airport.add_to(mymap)
mymap

In [ ]:
for lat, lng, airport in zip(df['lat'], df['lng'], df['airport']):
    airport = folium.Marker(
        location = [lat, lng],
        popup = airport,
        icon = folium.Icon(color = 'lightgray',    # icon to display in
                           icon = 'plane-arrival', # the marker
                           prefix = 'fa'),
    )
    airport.add_to(mymap)
mymap

In [ ]:
import pandas as pd

df = pd.read_csv('./datasets/flights/airports.csv')

In [ ]:
%pip install shapely

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.1 MB/s  0:00:00 eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from shapely.geometry import Point

df['geometry'] = df.apply(
    lambda row: Point(row['LONGITUDE'], row['LATITUDE']).wkt, axis=1)
df

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE,geometry
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040,POINT (-75.4404 40.65236)
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190,POINT (-99.6819 32.41132)
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919,POINT (-106.60919 35.04022)
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183,POINT (-98.42183 45.44906)
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447,POINT (-84.19447 31.53552)
...,...,...,...,...,...,...,...,...
317,WRG,Wrangell Airport,Wrangell,AK,USA,56.48433,-132.36982,POINT (-132.36982 56.48433)
318,WYS,Westerly State Airport,West Yellowstone,MT,USA,44.68840,-111.11764,POINT (-111.11764 44.6884)
319,XNA,Northwest Arkansas Regional Airport,Fayetteville/Springdale/Rogers,AR,USA,36.28187,-94.30681,POINT (-94.30681 36.28187)
320,YAK,Yakutat Airport,Yakutat,AK,USA,59.50336,-139.66023,POINT (-139.66023 59.50336)


In [ ]:
conn.execute("CREATE TABLE airports_2 AS SELECT * FROM df")

In [ ]:
conn.execute('INSTALL spatial;')
conn.execute('LOAD spatial;')

In [ ]:
conn.execute('''
  DROP TABLE IF EXISTS airports_2 ;
  CREATE TABLE airports_2 as
  SELECT
    *,
  ST_AsText(ST_Point(LONGITUDE,LATITUDE)) as geometry
  FROM airports
''')

In [ ]:
%pip install leafmap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.1/588.1 kB 5.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 874.0/874.0 kB 29.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 37.1 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 38.2 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 37.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 45.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 33.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 49.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.8 MB/s  0:00:00
   ━━━╺━

In [ ]:
%pip install mapclassify
%pip install geopandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.2/882.2 kB 4.1 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import leafmap
df_airports_gdf = leafmap.df_to_gdf(
    conn.execute('SELECT * FROM airports_2').df(),
    geometry = 'geometry',
    src_crs = "EPSG:4326",
    dst_crs = "EPSG:4326")

In [ ]:
# df_airports_gdf.explore()

## Finding nearby airports

In [ ]:
conn.execute('INSTALL spatial;')
conn.execute('LOAD spatial;')

# miami
LOCATION_LNGLAT = (-80.2706578, 25.7824017)

# within 3 degrees
df_airports_near_miami = conn.sql(f"""
    SELECT *
    FROM airports_2
    WHERE ST_DWithin(
      ST_GeomFromText(geometry),
      ST_GeomFromText('POINT ({LOCATION_LNGLAT[0]} {LOCATION_LNGLAT[1]})'),
      3);
""").df()

df_airports_near_miami

In [ ]:
conn.execute('INSTALL spatial;')
conn.execute('LOAD spatial;')
# miami
LOCATION_LNGLAT = (-80.2706578, 25.7824017)
# within 2 degrees
df_airports_near_miami = conn.sql(f"""
    SELECT *
    FROM airports_2
    WHERE ST_DWithin(
      ST_GeomFromText(geometry),
      ST_GeomFromText('POINT ({LOCATION_LNGLAT[0]} {LOCATION_LNGLAT[1]})'), 
      2);
""").df()

df_airports_near_miami

In [ ]:
df_airports_near_miami = conn.sql(f"""
    SELECT *,
      ST_Distance(ST_GeomFromText(geometry),
      ST_GeomFromText('POINT (
        {LOCATION_LNGLAT[0]}
        {LOCATION_LNGLAT[1]})')) as distance
    FROM airports_2;
""").df()

df_airports_near_miami

In [ ]:
df_airports_near_miami = conn.sql(f"""
    SELECT *,
      ST_Distance(ST_GeomFromText(geometry),
      ST_GeomFromText('POINT (
        {LOCATION_LNGLAT[0]}
        {LOCATION_LNGLAT[1]})')) as distance
    FROM airports_2
    ORDER by distance
    LIMIT 3
""").df()

df_airports_near_miami

In [ ]:
import leafmap

df_airports_near_miami_gdf = leafmap.df_to_gdf(df_airports_near_miami)
folium_map = df_airports_near_miami_gdf.explore()
folium_map

In [ ]:
import folium

# add a popup at Miami
folium.Marker(location = [LOCATION_LNGLAT[1],LOCATION_LNGLAT[0]],
              popup='Miami').add_to(folium_map)
folium_map

## Performing Descriptive Analytics

### Finding the Aireports for Each State and City

In [ ]:
df_city_state = conn.execute('''
  SELECT
     *
  FROM airports
  ORDER BY STATE, CITY
''').df()
df_city_state

In [ ]:
df_city_state.set_index(['STATE','CITY'], inplace = True)
df_city_state

In [ ]:
df_city_state.loc['CA']

In [ ]:
df_city_state.loc[('CA','San Francisco')]

### Aggregating the Total Number of Airports in Each State

In [ ]:
df_airports_state = conn.execute('''
  SELECT
    STATE,
    count(*) as COUNT
  FROM airports
  GROUP BY STATE
  ORDER BY Count DESC
''').df()
df_airports_state.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn

palette_color = seaborn.color_palette('pastel')

plt.figure(figsize = (7, 7))
plt.pie(df_airports_state['COUNT'],
        labels = df_airports_state['STATE'],
        colors = palette_color,
        autopct = '%.0f%%',)

plt.legend(df_airports_state['STATE'], loc = "best")

In [ ]:
df_airports_state = conn.execute('''
  SELECT
    STATE,
    count(*) as COUNT
  FROM airports
  GROUP BY STATE
  ORDER BY Count DESC
  LIMIT 10
''').df()
df_airports_state.head()

In [ ]:
# total number of airports
total = df_airports_state['COUNT'].sum()

def fmt(x):
    return '{:.1f}%\n({:.0f} airports)'.format(x, total * x / 100)

palette_color = seaborn.color_palette('pastel')

plt.figure(figsize = (7, 7))
plt.pie(df_airports_state['COUNT'],
        labels = df_airports_state['STATE'],
        colors = palette_color,
        autopct = fmt)

plt.legend(df_airports_state['STATE'], loc = "best")

In [ ]:
# create the bar chart
plt.bar(df_airports_state['STATE'],
        df_airports_state['COUNT'],
        color='skyblue')

plt.xlabel('State')
plt.ylabel('Number of Airports')
plt.title('Top 10 States with Most Airports')
plt.xticks(rotation = 45)

### Obtaining the Flight Counts for Each Pair of Origin and Destinatoin Airports

In [ ]:
conn.execute('''
  SELECT
    ORIGIN_AIRPORT, COUNT(ORIGIN_AIRPORT) as COUNT
  FROM flights
  GROUP BY ORIGIN_AIRPORT
  ORDER BY COUNT DESC
''').df()

In [ ]:
conn.execute('''
  SELECT
    ORIGIN_AIRPORT, DESTINATION_AIRPORT,
    COUNT(*) as COUNT
  FROM flights
  GROUP BY ORIGIN_AIRPORT, DESTINATION_AIRPORT
  ORDER BY COUNT DESC
''').df()

In [ ]:
conn.execute('''
  SELECT
    ORIGIN_AIRPORT, DESTINATION_AIRPORT,
    COUNT(*) as COUNT
  FROM flights
  WHERE AIRLINE='DL'
  GROUP BY ORIGIN_AIRPORT, DESTINATION_AIRPORT
  ORDER BY COUNT DESC
''').df()

### Getting the Canceled Flights from Airlines

In [ ]:
conn.execute('''
  SELECT
    ORIGIN_AIRPORT, DESTINATION_AIRPORT,
    COUNT(*) as COUNT
  FROM flights
  WHERE AIRLINE='DL' AND CANCELLED = 1
  GROUP BY ORIGIN_AIRPORT, DESTINATION_AIRPORT
  ORDER BY COUNT DESC
'''
).df()

In [ ]:
conn.execute('''
  SELECT
    ORIGIN_AIRPORT, DESTINATION_AIRPORT,
    (SUM(CANCELLED) * 100.0) / COUNT(*) as CANCELLED_PERCENT
  FROM flights
  WHERE AIRLINE = 'DL'
  GROUP BY ORIGIN_AIRPORT, DESTINATION_AIRPORT
  ORDER BY CANCELLED_PERCENT DESC
'''
).df()

In [ ]:
conn.execute('''
  SELECT
    (SUM(CANCELLED) * 100.0) / COUNT(*) as CANCELLED_PERCENT
  FROM flights
  WHERE AIRLINE = 'DL'
'''
).df()

In [ ]:
import matplotlib.pyplot as plt
df = conn.execute('''
  SELECT AIRLINE,
    (SUM(CANCELLED) * 100.0) / COUNT(*) as CANCELLED_PERCENT
  FROM flights
     -- WHERE AIRLINE = 'DL'
  GROUP BY AIRLINE
  ORDER BY CANCELLED_PERCENT DESC
'''
).df()
display(df)

In [ ]:
df.plot(kind='bar', x='AIRLINE', y='CANCELLED_PERCENT')

plt.xlabel('Airlines')
plt.ylabel('Cancellation Percentage')
plt.title('Cancellation Percentage for Different Airlines')

In [ ]:
import matplotlib.pyplot as plt
df = conn.execute('''
  SELECT
    a.AIRLINE,
    (SUM(f.CANCELLED) * 100.0) / COUNT(*) as CANCELLED_PERCENT
  FROM flights f
  JOIN airlines a ON f.AIRLINE = a.IATA_CODE
  GROUP BY a.AIRLINE
  ORDER BY CANCELLED_PERCENT DESC
'''
).df()
df.plot(kind='bar', x='AIRLINE', y='CANCELLED_PERCENT')
plt.xlabel('Airlines')
plt.ylabel('Cancellation Percentage')
plt.title('Cancellation Percentage for Different Airlines')

In [ ]:
ax = df.plot(kind='pie',
             x='AIRLINE',
             y='CANCELLED_PERCENT',
             labels = df['AIRLINE'],
             autopct = '%.0f%%',
             legend=False
)
ax.get_yaxis().set_visible(False)
plt.xlabel('Airlines')
plt.title('Cancellation Percentage for Different Airlines')

### Getting the Flight Count for Each Day of the Week

In [ ]:
df_flights_day_of_week = conn.execute('''
  SELECT
    day_of_week,
    origin_airport,
    destination_airport,
    COUNT(*) AS flight_count
  FROM
    flights
  WHERE
    CANCELLED = 0 -- Exclude cancelled flights
  GROUP BY
    day_of_week,
    origin_airport,
    destination_airport
  ORDER BY
    day_of_week,
    origin_airport,
    destination_airport;
''').df()
df_flights_day_of_week

In [ ]:
conn.execute('''
  SELECT 
    YEAR, MONTH, DAY, ORIGIN_AIRPORT, DESTINATION_AIRPORT
  FROM flights
  WHERE ORIGIN_AIRPORT NOT SIMILAR TO '[A-Za-z]+'
  OR DESTINATION_AIRPORT NOT SIMILAR TO '[A-Za-z]+';
''').df()

In [ ]:
from_airport = 'SFO'
to_airport = 'LAX'

df_flights_result = df_flights_day_of_week.query(
    f'ORIGIN_AIRPORT=="{from_airport}" & DESTINATION_AIRPORT=="{to_airport}"')
df_flights_result.plot(kind='bar',
                       x = 'DAY_OF_WEEK',
                       y = 'flight_count',
                       legend = False)

plt.xlabel('Day of Week')
plt.ylabel('Number of Flights')
plt.title(f'Number of Flights from {from_airport} to {to_airport}')

plt.xticks(df_flights_result['DAY_OF_WEEK'] - 1,
           ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
plt.show()

In [ ]:
df_flights_weekday = conn.execute('''
with t1 as (
  SELECT
    day_of_week,
    origin_airport,
    destination_airport,
    COUNT(*) AS flight_count_per_weekday
  FROM
    flights
  WHERE
    CANCELLED = 0 -- Exclude cancelled flights
  GROUP BY
    day_of_week,
    origin_airport,
    destination_airport
  ORDER BY
    day_of_week,
    origin_airport,
  destination_airport
),
t2 as (
  SELECT
    origin_airport,
    destination_airport,
    count(*) as total_flight_count
  FROM
    flights
  WHERE
    CANCELLED = 0 -- Exclude cancelled flights
  GROUP BY
    origin_airport,
    destination_airport
)
SELECT
  t1.origin_airport,
  t1.destination_airport,
  t1.day_of_week,
  t2.total_flight_count,
  100. * (t1.flight_count_per_weekday / t2.total_flight_count) as
  percent_flights_on_weekday
FROM t1
JOIN t2
ON
  t1.origin_airport = t2.origin_airport AND
  t1.destination_airport = t2.destination_airport
''').df()
df_flights_weekday

In [ ]:
from_airport = 'SFO'
to_airport = 'LAX'

# step 1: Filter the DataFrame for SFO to LAX
sfo_to_las_flights = df_flights_weekday.query(
f'ORIGIN_AIRPORT=="{from_airport}" & DESTINATION_AIRPORT=="{to_airport}"')

# step 2: Plotting
plt.figure(figsize=(10, 6))
plt.bar(sfo_to_las_flights['DAY_OF_WEEK'],
        sfo_to_las_flights['percent_flights_on_weekday'],
        color='skyblue')

plt.xticks(sfo_to_las_flights['DAY_OF_WEEK'].sort_values(),
           ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
plt.title(f'Percentage of Flights from {from_airport} ' + 
          f'to {to_airport} by Day of the Week')

plt.xlabel('Day of the Week')
plt.ylabel('Percentage of Flights')
plt.xticks(rotation=45)
plt.grid(axis='y')

### Finding the Most Common Timeslot for Flight Delays

In [ ]:
df_delays_by_week = conn.execute('''
SELECT
  DAY_OF_WEEK,
CASE
  WHEN SCHEDULED_DEPARTURE BETWEEN '0000' AND '0559' THEN '00:00-06:00'
  WHEN SCHEDULED_DEPARTURE BETWEEN '0600' AND '1159' THEN '06:00-12:00'
  WHEN SCHEDULED_DEPARTURE BETWEEN '1200' AND '1759' THEN '12:00-18:00'
  WHEN SCHEDULED_DEPARTURE BETWEEN '1800' AND '2400' THEN '18:00-24:00'
ELSE 'Other'
  END AS DEPARTURE_TIME_INTERVAL,
  AVG(ARRIVAL_DELAY) AS AVG_ARRIVAL_DELAY
FROM
  flights
WHERE
  ARRIVAL_DELAY > 0
GROUP BY
  DAY_OF_WEEK,
CASE
  WHEN SCHEDULED_DEPARTURE BETWEEN '0000' AND '0559' THEN '00:00-06:00'
  WHEN SCHEDULED_DEPARTURE BETWEEN '0600' AND '1159' THEN '06:00-12:00'
  WHEN SCHEDULED_DEPARTURE BETWEEN '1200' AND '1759' THEN '12:00-18:00'
  WHEN SCHEDULED_DEPARTURE BETWEEN '1800' AND '2400' THEN '18:00-24:00'
ELSE 'Other'
END
ORDER BY
  DAY_OF_WEEK, DEPARTURE_TIME_INTERVAL;
''').df()
df_delays_by_week

In [ ]:
df_delays_by_week_pivot = df_delays_by_week.pivot(
index = 'DAY_OF_WEEK',
columns = 'DEPARTURE_TIME_INTERVAL',
values = 'AVG_ARRIVAL_DELAY')
df_delays_by_week_pivot

In [ ]:
# plotting a bar chart
df_delays_by_week_pivot.plot(kind='bar',
                             stacked=False,
figsize=(10, 6))

# updating the x-ticks to show the days of the week
days_of_week = ['Monday', 'Tuesday', 'Wednesday',
                'Thursday', 'Friday', 'Saturday', 'Sunday']

plt.xticks(ticks=range(len(days_of_week)), labels=days_of_week, rotation=0)
plt.title('Average Arrival Delay by Departure Time and Day of Week')
plt.xlabel('Day of Week')
plt.ylabel('Average Arrival Delay (minutes)')
plt.legend(title='Departure Time Interval', bbox_to_anchor=(1, 1))
plt.tight_layout() # Adjust layout to avoid clipping of labels
plt.show()

### Finding the Airlines with the Most and Fewest Delays

In [ ]:
df_most_delays = conn.execute('''
  SELECT
    count(airlines.AIRLINE) as Count,
    airlines.AIRLINE
  FROM flights, airlines
  WHERE airlines.IATA_CODE = flights.AIRLINE AND flights.ARRIVAL_DELAY > 0
  GROUP BY airlines.AIRLINE
  ORDER BY COUNT DESC
''').df()
df_most_delays

In [ ]:
df_percent_delay = conn.execute('''
WITH flight_delays AS (
  SELECT
  AIRLINE,
  1.0 * count(*) as TotalFlights,
  1.0 * sum(case when ARRIVAL_DELAY > 0 then 1 else 0 end) as Delays,
  (1.0 * sum(case when ARRIVAL_DELAY > 0 then 1 else 0 end) /
  count(*)) * 100 as Percentage
FROM flights
GROUP BY AIRLINE
)
SELECT
  flight_delays.Percentage,
  airlines.IATA_CODE,
  airlines.AIRLINE
FROM flight_delays
  JOIN airlines ON airlines.IATA_CODE = flight_delays.AIRLINE
ORDER BY flight_delays.Percentage DESC;
''').df()

In [ ]:
df_percent_delay

In [ ]:
plt.bar(df_percent_delay['AIRLINE'],
df_percent_delay['Percentage'],
        color='skyblue')

plt.title('Percentage of Delayed Flights by Airline')
plt.xlabel('Airlines')
plt.ylabel('Percentage of Delayed Flights')
plt.xticks(rotation = 90)
plt.grid(axis='y')

In [ ]:
df_percent_on_time = conn.execute('''
WITH flight_delays AS (
  SELECT
    AIRLINE,
    1.0 * count(*) as TotalFlights,
    1.0 * sum(case when ARRIVAL_DELAY <= 0 then 1 else 0 end) as
    OnTimeFlights,
    (1.0 * sum(case when ARRIVAL_DELAY <= 0 then 1 else 0 end) /
    count(*)) * 100 as Percentage
  FROM flights
  GROUP BY AIRLINE
)
SELECT
  flight_delays.Percentage,
  airlines.IATA_CODE,
  airlines.AIRLINE
FROM flight_delays
JOIN airlines ON airlines.IATA_CODE = flight_delays.AIRLINE
ORDER BY flight_delays.Percentage DESC;
''').df()
df_percent_on_time

In [ ]:
plt.bar(df_percent_on_time['AIRLINE'],
df_percent_on_time['Percentage'],
        color='skyblue')

plt.title('Percentage of Ontime Flights by Airline')
plt.xlabel('Airlines')
plt.ylabel('Percentage of Flights Ontime')
plt.xticks(rotation = 90)
plt.grid(axis='y')